In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Ethnicity

In [48]:
def compute_hit_rate_ethnicity(path, county, city):    
    df = pd.read_csv(path, compression = 'zip')
    df_searched = df.query("search_conducted == True")
    df_searched["date"] = pd.to_datetime(df_searched["date"])
    df_searched["year"] = df_searched["date"].dt.year
    df_searched_cleaned = df_searched[['year', 'subject_race', 'search_conducted', 'contraband_found']]
    found = pd.DataFrame(df_searched_cleaned.groupby(['year', 'subject_race'])['contraband_found'].value_counts()).rename(columns={'contraband_found' : 'nb_find'}).reset_index()
    all = pd.DataFrame(df_searched_cleaned.groupby(['year', 'subject_race'])['search_conducted'].value_counts()).rename(columns={'search_conducted' : 'nb_search'}).reset_index()
    found_and_all = pd.merge(left = found, right = all, left_on=['year','subject_race'], right_on = ['year', 'subject_race']).drop(['search_conducted'], axis = 1)
    found_and_all['Hit_rate'] = found_and_all['nb_find'] / found_and_all['nb_search'] * 100
    found_and_all['County'] = county
    found_and_all['City'] = city
    return found_and_all.query('contraband_found == True')

In [50]:
all_city_ethnicity = compute_hit_rate_ethnicity('..\data\city\ca_oakland.csv.zip', 'California', 'Oakland')

Done on all the file

In [69]:
new_df_ethnicity = compute_hit_rate_ethnicity('../data/city/tx_san_antonio.csv.zip', 'Texas', 'San Antonio')

In [70]:
all_city_ethnicity = pd.concat([all_city_ethnicity, new_df_ethnicity])

In [72]:
all_city_ethnicity.to_csv('../data/city/all_hit_rate_ethnicity')

In [74]:
all_city

,year,subject_race,contraband_found,nb_find,nb_search,Hit_rate,County,City
16,2016.0,asian/pacific islander,True,110,292,37.671233,California,Oakland
18,2016.0,black,True,2608,7350,35.482993,California,Oakland
20,2016.0,hispanic,True,634,1633,38.824250,California,Oakland
22,2016.0,other,True,55,180,30.555556,California,Oakland
24,2016.0,white,True,239,583,40.994854,California,Oakland
...,...,...,...,...,...,...,...,...
61,2018.0,asian/pacific islander,True,1,1,100.000000,Texas,San Antonio
63,2018.0,black,True,6,50,12.000000,Texas,San Antonio
65,2018.0,hispanic,True,9,151,5.960265,Texas,San Antonio
67,2018.0,other,True,2,10,20.000000,Texas,San Antonio


# Gender

In [75]:
def compute_hit_rate_ethnicity(path, county, city):    
    df = pd.read_csv(path, compression = 'zip')
    df_searched = df.query("search_conducted == True")
    df_searched["date"] = pd.to_datetime(df_searched["date"])
    df_searched["year"] = df_searched["date"].dt.year
    df_searched_cleaned = df_searched[['year', 'subject_sex', 'search_conducted', 'contraband_found']]
    found = pd.DataFrame(df_searched_cleaned.groupby(['year', 'subject_sex'])['contraband_found'].value_counts()).rename(columns={'contraband_found' : 'nb_find'}).reset_index()
    all = pd.DataFrame(df_searched_cleaned.groupby(['year', 'subject_sex'])['search_conducted'].value_counts()).rename(columns={'search_conducted' : 'nb_search'}).reset_index()
    found_and_all = pd.merge(left = found, right = all, left_on=['year','subject_sex'], right_on = ['year', 'subject_sex']).drop(['search_conducted'], axis = 1)
    found_and_all['Hit_rate'] = found_and_all['nb_find'] / found_and_all['nb_search'] * 100
    found_and_all['County'] = county
    found_and_all['City'] = city
    return found_and_all.query('contraband_found == True')

In [76]:
all_city_gender = compute_hit_rate_gender('..\data\city\ca_oakland.csv.zip', 'California', 'Oakland')

In [90]:
new_df_gender = compute_hit_rate_gender('../data/city/tx_san_antonio.csv.zip', 'Texas', 'San Antonio')

In [91]:
all_city_gender = pd.concat([all_city_gender, new_df_gender])

In [92]:
all_city_gender

,year,subject_sex,contraband_found,nb_find,nb_search,Hit_rate,County,City
7,2016.0,female,True,594,1734,34.256055,California,Oakland
9,2016.0,male,True,3051,8303,36.745755,California,Oakland
11,2017.0,female,True,392,1876,20.895522,California,Oakland
13,2017.0,male,True,2114,9005,23.475847,California,Oakland
1,2014.0,female,True,101,1326,7.616893,California,San Diego
...,...,...,...,...,...,...,...,...
19,2016.0,male,True,66,577,11.438475,Texas,San Antonio
21,2017.0,female,True,30,213,14.084507,Texas,San Antonio
23,2017.0,male,True,110,881,12.485812,Texas,San Antonio
25,2018.0,female,True,8,98,8.163265,Texas,San Antonio


In [93]:
all_city_gender.to_csv('../data/city/all_hit_rate_gender.csv')

# Mean By state

In [58]:
all_city_gender_mean = pd.read_csv('../data/city/all_hit_rate_gender.csv')
all_city_ethnicity_mean = pd.read_csv('../data/city/all_hit_rate_ethnicity.csv')

In [53]:
all_city_gender_mean_hit_rate = all_city_gender_mean.groupby(['year', 'subject_sex', 'County'])['Hit_rate'].agg(['mean']).reset_index()
all_city_gender_mean_search_count = all_city_gender_mean.groupby(['year', 'subject_sex'])['nb_search'].agg(['sum']).reset_index()
all_city_gender_mean = pd.merge(all_city_gender_mean_hit_rate, all_city_gender_mean_search_count, left_on=['year', 'subject_sex'], right_on=['year', 'subject_sex'])

In [54]:
all_city_gender_mean.to_csv('../data/city/all_hit_rate_gender_mean.csv')

In [59]:
all_city_ethnicity_mean_hit_rate = all_city_ethnicity_mean.groupby(['year', 'subject_race', 'County'])['Hit_rate'].agg(['mean']).reset_index()
all_city_ethnicity_mean_search_count = all_city_ethnicity_mean.groupby(['year', 'subject_race'])['nb_search'].agg(['sum']).reset_index()

all_city_ethnicity_mean = pd.merge(all_city_ethnicity_mean_hit_rate, all_city_ethnicity_mean_search_count, left_on=['year', 'subject_race'], right_on=['year', 'subject_race'])

TypeError: unhashable type: 'list'

In [56]:
all_city_ethnicity_mean.to_csv('../data/city/all_hit_rate_ethnicity_mean.csv')